We already have patent_df which is perfectly filtered to scores > 3 and also these IDs are unique(which means there are no duplicates) and also been labelled as 1 


This is considered as confirmed patentable science

Now we focus only on the negatively labelled dataset (label=0)

In brief 
1).No known patent linkage
2).never been patented

In [1]:
import pandas as pd

We drop duplicates of labels and paperid

In [2]:
positive_df = pd.read_csv(r"A:/DLSA/Project_work/Datasets/patent_df.csv").iloc[:, 1:]
positive_df = positive_df[["paperid", "label"]].drop_duplicates("paperid")

the labelling comes from works.csv dataset

In [3]:
works = pd.read_csv(r"A:/DLSA/Project_work/Datasets/clean_works1.csv", low_memory=False).iloc[:, 1:]
works = works.drop_duplicates("paperid")

We merge the patents dataset which has labelled =1 with the works dataset

In [4]:
pos_with_year = positive_df.merge(works[["paperid", "publication_year"]],on="paperid",how="left")

In [5]:
pos_with_year

,paperid,label,publication_year
0,W4286853979,1,2013.0
1,W2086685806,1,2012.0
2,W2006576711,1,2013.0
3,W1986737941,1,2008.0
4,W2066224895,1,2013.0
...,...,...,...
190411,W4290887410,1,2014.0
190412,W4304731564,1,2022.0
190413,W2735055015,1,2017.0
190414,W2550033686,1,2015.0


In [6]:
years_valid = pos_with_year["publication_year"].dropna().unique()
works_years = works[works["publication_year"].isin(years_valid)].copy()

We do a check because if the older paper can be patented in the future then we need to identify them too otherwise all will be concluded as
negative labels

In [7]:
neg_pool = works_years[~works_years["paperid"].isin(positive_df["paperid"])].copy()

In [8]:
n_pos = len(positive_df)
n_neg = len(neg_pool)

print("Negative : Positive ratio =", round(n_neg / n_pos, 2))

print(n_pos)
print(n_neg)


Negative : Positive ratio = 2.99
190416
568954


In [9]:
tradeoff = 3
n_pos = len(positive_df)
n_neg = min(len(neg_pool), n_pos * tradeoff)

We simply label the rest of the papers as label =0 which shows it is negative

In [10]:
neg = neg_pool.sample(n=n_neg, random_state=42)[["paperid"]].drop_duplicates("paperid")
neg["label"] = 0

concat both the positive labelled ones and negative dataset

In [11]:
sampled = pd.concat([positive_df, neg], ignore_index=True).drop_duplicates("paperid")

In [12]:
sampled = sampled.sample(frac=1, random_state=42).reset_index(drop=True)

In [13]:
sampled.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 759370 entries, 0 to 759369
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   paperid  759370 non-null  object
 1   label    759370 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 11.6+ MB


In [40]:
print(sampled["label"].value_counts())

label
0    568954
1    190416
Name: count, dtype: int64


In [41]:
sampled.to_csv("A:/DLSA/Project_work/Datasets/negative_sampling1.csv")